# Assignment 1 for Clustering: Target Trial Emulation
- New and novel methods in Machine Learning are made either by borrowing formulas and concepts from other scientific fields and redefining it based on new sets of assumptions, or by adding an extra step to an already existing framework of methodology.

- In this exercise (Assignment 1 of the Clustering Topic), we will try to develop a novel method of Target Trial Emulation by integrating concepts of Clustering into the already existing framework. Target Trial Emulation is a new methodological framework in epidemiology which tries to account for the biases in old and traditional designs.

These are the instructions:
1. Look at this website: https://rpubs.com/alanyang0924/TTE
2. Extract the dummy data in the package and save it as "data_censored.csv"
2. Convert the R codes into Python Codes (use Jupyter Notebook), replicate the results using your python code.
3. Create another copy of your Python Codes, name it TTE-v2 (use Jupyter Notebook).
4. Using TTE-v2, think of a creative way on where you would integrate a clustering mechanism, understand each step carefully and decide at which step a clustering method can be implemented. Generate insights from your results.
5. Do this by pair, preferably your thesis partner.
6. Push to your github repository.
7. Deadline is: February 28, 2025 at 11:59 pm.

## I. Necessary Imports

In [12]:
import pandas as pd
import numpy as np
import os
import patsy
import joblib
import json
from sklearn.linear_model import LogisticRegression
from IPython.display import display
import statsmodels.api as sm
import statsmodels.formula.api as smf




## II. Class Definition and Required Functions

In [13]:
def stats_glm_logit(save_path):
    if save_path is not None:
        os.makedirs(save_path, exist_ok=True)

    def fit_model(numerator, denominator, data):
        # Fit model using statsmodels (instead of sklearn)
        formula = f"treatment ~ {denominator}"  
        model = smf.logit(formula, data).fit()

        # Save model
        model_path = os.path.join(save_path, "logit_model.pkl")
        joblib.dump(model, model_path)

        model_details = {
            "numerator": numerator, 
            "denominator": denominator,
            "model_type": "te_stats_glm_logit",
            "file_path": model_path
        }
        json.dump(model_details, open(os.path.join(save_path, "model_details.json"), "w"))

        return model
    
    return fit_model


class TrialSequence:
    def __init__(self, esitmand, **kwargs):
        self.estimand = esitmand
        self.data = None
        self.censor_weights = None
        self.switch_weights = None
        self.outcome_model = None
        self.expansion = None
        self.outcome_data = None

    def set_data(self, data):
        self.data = data

    def show(self):
        print(f"Trial Sequence Object\nEstimand: {self.estimand}\n")
        
        if self.data is not None:
            display(self.data)
        else:
            print("No data set")

        print("\nIPW for informative censoring:")
        print(self.censor_weights if self.censor_weights is not None else "Not calculated.")
        if self.switch_weights is not None:
            print("\nIPW for treatment switch censoring:")
            print(self.switch_weights)
            
        print("\nOutcome model:")
        print(self.outcome_model if self.outcome_model is not None else "Not specified.")
        if self.outcome_data is not None:
            print("\nOutcome data:")
            print(self.outcome_data)

#STEP 3
    
    def set_switch_weight_model(self, numerator=None, denominator=None, model_fitter=None, eligible_wts_0=None, eligible_wts_1=None):
        if self.data is None:
            raise ValueError("set_data() before setting switch weight models")
        
        if self.estimand == "ITT":
            raise ValueError("Switching weights are not supported for intention-to-treat analyses")

        if eligible_wts_0 and eligible_wts_0 in self.data.columns:
            self.data = self.data.rename(columns={eligible_wts_0: "eligible_wts_0"})
        if eligible_wts_1 and eligible_wts_1 in self.data.columns:
            self.data = self.data.rename(columns={eligible_wts_1: "eligible_wts_1"})

        if numerator is None:
            numerator = "1"
        if denominator is None:
            denominator = "1"
        
        if "time_on_regime" in denominator:
            raise ValueError("time_on_regime should not be used in denominator.")

        formula_numerator = f"treatment ~ {numerator}"
        formula_denominator = f"treatment ~ {denominator}"

        self.switch_weights = {
            "numerator": formula_numerator,
            "denominator": formula_denominator,
            "model_fitter": "te_stats_glm_logit",
        }

        if model_fitter is not None:
            fitted_model = model_fitter(numerator, denominator, self.data)  
            self.switch_weights["fitted_model"] = fitted_model 

    def show_switch_weights(self):
        return self.switch_weights if self.switch_weights else "Not calculated"
    
    def show_censor_weights(self):
        return self.censor_weights if self.censor_weights else "Not calculated"
    

    def set_censor_weight_model(self, censor_event, numerator="1", denominator="1", pool_models="none", model_fitter=None):
        if model_fitter is None: 
            model_fitter = stats_glm_logit()

        if censor_event not in self.data.columns:
            raise ValueError(f"'{censor_event}' must be a column in the dataset.")

        formula_numerator = f"1 - {censor_event} ~ {numerator}"
        formula_denominator = f"1 - {censor_event} ~ {denominator}"

        self.censor_weights = {
            "numerator": formula_numerator,
            "denominator": formula_denominator,
            "pool_numerator": pool_models in ["numerator", "both"],
            "pool_denominator": pool_models == "both",
            "model_fitter": "te_stats_glm_logit"
        }

        # Fit the numerator model using statsmodels
        self.censor_weights["fitted_model"] = model_fitter(numerator, denominator, self.data)

        # Fit separate denominator models if not pooling
        if not self.censor_weights["pool_denominator"]:
            self.censor_weights["fitted_model_0"] = model_fitter(numerator, denominator, self.data[self.data["previous_treatment"] == 0])
            self.censor_weights["fitted_model_1"] = model_fitter(numerator, denominator, self.data[self.data["previous_treatment"] == 1])


    
#STEP 4
    def calculate_weights(self, quiet=False):
        use_censor_weights = isinstance(self.censor_weights, dict) and self.censor_weights.get("fitted_model") is not None

        if self.estimand == "PP":
            if not (isinstance(self.switch_weights, dict) and self.switch_weights.get("fitted_model")):
                raise ValueError("Switch weight models are not specified. Use set_switch_weight_model()")
            self._calculate_weights_trial_seq(quiet, switch_weights=True, censor_weights=use_censor_weights)
        elif self.estimand == "ITT":
            self._calculate_weights_trial_seq(quiet, switch_weights=False, censor_weights=use_censor_weights)
        else:
            raise ValueError(f"Unknown estimand: {self.estimand}")


    def _calculate_weights_trial_seq(self, quiet, switch_weights, censor_weights):
        if switch_weights:
            if not quiet:
                print("Calculating switch weights...")
            
            switch_model = self.switch_weights["fitted_model"]
            self.data["switch_prob"] = switch_model.predict(self.data)
            self.data["switch_weight"] = 1 / self.data["switch_prob"]

        if censor_weights:
            if not quiet:
                print("Calculating censor weights...")

            censor_model = self.censor_weights["fitted_model"]
            self.data["censor_prob"] = censor_model.predict(self.data)
            self.data["censor_weight"] = 1 / self.data["censor_prob"]

        # Compute final weight
        if switch_weights and censor_weights:
            self.data["final_weight"] = self.data["switch_weight"] * self.data["censor_weight"]
        elif switch_weights:
            self.data["final_weight"] = self.data["switch_weight"]
        elif censor_weights:
            self.data["final_weight"] = self.data["censor_weight"]



    def show_weight_models(self):
        if "censored" not in self.data.columns:
            raise ValueError("Column 'censored' not found in dataset.")

        # Convert boolean censored to integer if necessary
        self.data["censored"] = self.data["censored"].astype(int)

        # Create explicit column for 1 - censored
        self.data["censored_inv"] = 1 - self.data["censored"]

        if self.censor_weights is not None:
            print("## Weight Models for Informative Censoring")
            print("#\n#")

            # Numerator model
            print("## Model: P(censored = 0 | X) for numerator")
            print("#")
            numerator_model = smf.logit("censored_inv ~ x1 + x2", data=self.data).fit(method="newton")
            print(numerator_model.summary2().tables[1].round(6))
            print("#")

            # Denominator models
            if self.censor_weights["pool_denominator"]:
                print("## Model: P(censored = 0 | X) for denominator")
                print("#")
                denominator_model = smf.logit("censored_inv ~ x1 + x2", data=self.data).fit(method="newton")
                print(denominator_model.summary2().tables[1].round(6))
            else:
                print("## Model: P(censored = 0 | X, previous treatment = 0) for denominator")
                print("#")
                denominator_model_0 = smf.logit("censored_inv ~ x1 + x2", 
                                                data=self.data[self.data["previous_treatment"] == 0]).fit(method="newton")
                print(denominator_model_0.summary2().tables[1].round(6))
                print("#")

                print("## Model: P(censored = 0 | X, previous treatment = 1) for denominator")
                print("#")
                denominator_model_1 = smf.logit("censored_inv ~ x1 + x2", 
                                                data=self.data[self.data["previous_treatment"] == 1]).fit(method="newton")
                print(denominator_model_1.summary2().tables[1].round(6))

        if self.switch_weights is not None:
            print("## Weight Models for Treatment Switch")
            print("#\n#")

            # Numerator model
            print("## Model: P(switch = 1 | X) for numerator")
            print("#")
            numerator_model = smf.logit("treatment ~ age + x1 + x3", data=self.data).fit(method="newton")
            print(numerator_model.summary2().tables[1].round(6))
            print("#")

            # Denominator model
            print("## Model: P(switch = 1 | X) for denominator")
            print("#")
            denominator_model = smf.logit("treatment ~ age + x1 + x3", data=self.data).fit(method="newton")
            print(denominator_model.summary2().tables[1].round(6))




#STEP 5
    def set_outcome_model(self, adjustment_terms="1"):
        if self.data is None:
            raise ValueError("set_data() before defining the outcome model.")

        # Determine treatment variable (PP vs ITT)
        treatment_var = "treatment" if self.estimand in ["ITT", "PP"] else "dose"

        # Retrieve Stabilized Weight Terms
        stabilised_weight_terms = "1"
        if self.switch_weights:
            stabilised_weight_terms += " + " + " + ".join(self.switch_weights['numerator'].split("~")[1].strip().split(" + "))
        if self.censor_weights:
            stabilised_weight_terms += " + " + " + ".join(self.censor_weights['numerator'].split("~")[1].strip().split(" + "))

        # Check if followup_time and trial_period exist
        additional_terms = []
        if "followup_time" in self.data.columns:
            additional_terms.append("followup_time + I(followup_time**2)")
        if "trial_period" in self.data.columns:
            additional_terms.append("trial_period + I(trial_period**2)")

        additional_formula = " + ".join(additional_terms) if additional_terms else ""

        # Create regression formula dynamically
        formula = f"outcome ~ {treatment_var} + {adjustment_terms} + {stabilised_weight_terms}"
        if additional_formula:
            formula += " + " + additional_formula

        # 🚀 Ensure weights exist before fitting the model
        if "final_weight" not in self.data.columns:
            raise ValueError("Weights have not been calculated. Run calculate_weights() first.")

        # 🔹 Extract predictor variables from formula
        predictor_vars = formula.split("~")[1].strip().split(" + ")
        predictor_vars = [var.strip() for var in predictor_vars if var.strip() != "1"]  # ✅ Remove "1"

        # ✅ Corrected Logistic Regression with Weights using GLM
        model = sm.GLM(
            self.data["outcome"],  # Dependent variable
            sm.add_constant(self.data[predictor_vars]),  # Independent variables (only those used in training)
            family=sm.families.Binomial(),  # Logistic regression
            weights=self.data["final_weight"]  # Correct weight handling
        ).fit()

        # 🚀 **Fix: Ensure prediction uses the same features as training**
        self.data["predicted_outcome"] = model.predict(sm.add_constant(self.data[predictor_vars]))  # ✅ Corrected
        self.data["residuals"] = self.data["outcome"] - self.data["predicted_outcome"]  # Residuals

        # Store the outcome model
        self.outcome_model = model

        return model
        
    def show_outcome_model(self):
        if self.outcome_model is None:
            return "Outcome model not specified."
        return self.outcome_model.summary()


#Subclass of Trial Sequence, handles the PP (hehe) estimand
class TrialSequencePP(TrialSequence):
    def __init__(self, **kwargs):
        super().__init__("PP", **kwargs)
 
#Subclass of Trial Sequence, handles the ITT estimand
class TrialSequenceITT(TrialSequence):
    def __init__(self, **kwargs):
        super().__init__("ITT", **kwargs)

#trial_sequence function equivalent used in the article
def trial_sequence(estimand, **kwargs):
    estimand_classes = {
        "PP": TrialSequencePP,
        "ITT": TrialSequenceITT
    }

    if estimand not in estimand_classes:
        raise ValueError(f"{estimand} is not a valid estimand, choose either PP or ITT")
    
    return estimand_classes[estimand](**kwargs)

## III. Process

### 1. Setup
A sequence of target trials analysis starts by specifying which estimand will be used:

In [14]:
trial_pp = trial_sequence("PP")
trial_itt = trial_sequence("ITT")

### 2. Data Preparation
Next the user must specify the observational input data that will be used for the target trial emulation. Here we need to specify which columns contain which values and how they should be used.

In [15]:
data_censored = pd.read_csv("data_censored.csv")
print("Extracted Dummy Data")
display(data_censored)
data_censored["previous_treatment"] = data_censored["treatment"].shift(1).fillna(0)
#Setting the dataset to the data field
trial_pp.set_data(data_censored)
trial_itt.set_data(data_censored)

#Displaying the info stored in each class
trial_pp.show()
trial_itt.show()

Extracted Dummy Data


,id,period,treatment,x1,x2,x3,x4,age,age_s,outcome,censored,eligible
0,1,0,1,1,1.146148,0,0.734203,36,0.083333,0,0,1
1,1,1,1,1,0.002200,0,0.734203,37,0.166667,0,0,0
2,1,2,1,0,-0.481762,0,0.734203,38,0.250000,0,0,0
3,1,3,1,0,0.007872,0,0.734203,39,0.333333,0,0,0
4,1,4,1,1,0.216054,0,0.734203,40,0.416667,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
720,99,3,0,0,-0.747906,1,0.575268,68,2.750000,0,0,0
721,99,4,0,0,-0.790056,1,0.575268,69,2.833333,0,0,0
722,99,5,1,1,0.387429,1,0.575268,70,2.916667,0,0,0
723,99,6,1,1,-0.033762,1,0.575268,71,3.000000,0,0,0


Trial Sequence Object
Estimand: PP



,id,period,treatment,x1,x2,x3,x4,age,age_s,outcome,censored,eligible,previous_treatment
0,1,0,1,1,1.146148,0,0.734203,36,0.083333,0,0,1,0.0
1,1,1,1,1,0.002200,0,0.734203,37,0.166667,0,0,0,1.0
2,1,2,1,0,-0.481762,0,0.734203,38,0.250000,0,0,0,1.0
3,1,3,1,0,0.007872,0,0.734203,39,0.333333,0,0,0,1.0
4,1,4,1,1,0.216054,0,0.734203,40,0.416667,0,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
720,99,3,0,0,-0.747906,1,0.575268,68,2.750000,0,0,0,0.0
721,99,4,0,0,-0.790056,1,0.575268,69,2.833333,0,0,0,0.0
722,99,5,1,1,0.387429,1,0.575268,70,2.916667,0,0,0,0.0
723,99,6,1,1,-0.033762,1,0.575268,71,3.000000,0,0,0,1.0



IPW for informative censoring:
Not calculated.

Outcome model:
Not specified.
Trial Sequence Object
Estimand: ITT



,id,period,treatment,x1,x2,x3,x4,age,age_s,outcome,censored,eligible,previous_treatment
0,1,0,1,1,1.146148,0,0.734203,36,0.083333,0,0,1,0.0
1,1,1,1,1,0.002200,0,0.734203,37,0.166667,0,0,0,1.0
2,1,2,1,0,-0.481762,0,0.734203,38,0.250000,0,0,0,1.0
3,1,3,1,0,0.007872,0,0.734203,39,0.333333,0,0,0,1.0
4,1,4,1,1,0.216054,0,0.734203,40,0.416667,0,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
720,99,3,0,0,-0.747906,1,0.575268,68,2.750000,0,0,0,0.0
721,99,4,0,0,-0.790056,1,0.575268,69,2.833333,0,0,0,0.0
722,99,5,1,1,0.387429,1,0.575268,70,2.916667,0,0,0,0.0
723,99,6,1,1,-0.033762,1,0.575268,71,3.000000,0,0,0,1.0



IPW for informative censoring:
Not calculated.

Outcome model:
Not specified.


### 3. Weight Models
To adjust for the effects of informative censoring, inverse probability of censoring weights (IPCW) can be applied. To estimate these weights, we construct time-to-(censoring) event models. Two sets of models are fit for the two censoring mechanisms which may apply: censoring due to deviation from assigned treatment and other informative censoring.
#### 3.1 Censoring due to treatment switching
We specify model formulas to be used for calculating the probability of receiving treatment in the current period. Separate models are fitted for patients who had treatment = 1 and those who had treatment = 0 in the previous period. Stabilized weights are used by fitting numerator and denominator models.

There are optional arguments to specify columns which can include/exclude observations from the treatment models. These are used in case it is not possible for a patient to deviate from a certain treatment assignment in that period.

In [16]:
path = "Models"
trial_pp.set_switch_weight_model(numerator="age", denominator="age + x1 + x3", model_fitter=stats_glm_logit(save_path=os.path.join(path, "switch_models")))
trial_pp.show_switch_weights()

Optimization terminated successfully.
         Current function value: 0.660234
         Iterations 5


{'numerator': 'treatment ~ age',
 'denominator': 'treatment ~ age + x1 + x3',
 'model_fitter': 'te_stats_glm_logit',
 'fitted_model': <statsmodels.discrete.discrete_model.BinaryResultsWrapper at 0x1cdd31b7560>}

#### 3.2 Other informative censoring
In case there is other informative censoring occurring in the data, we can create similar models to estimate the IPCW. These can be used with all types of estimand. We need to specifycensor_event which is the column containing the censoring indicator.

In [17]:
trial_pp.set_censor_weight_model(censor_event="censored", numerator="x2", denominator="x2 + x1", pool_models="none", model_fitter=stats_glm_logit(save_path=os.path.join(path, "switch_models")))
trial_pp.show_censor_weights()

Optimization terminated successfully.
         Current function value: 0.682694
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.623353
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.634002
         Iterations 5


{'numerator': '1 - censored ~ x2',
 'denominator': '1 - censored ~ x2 + x1',
 'pool_numerator': False,
 'pool_denominator': False,
 'model_fitter': 'te_stats_glm_logit',
 'fitted_model': <statsmodels.discrete.discrete_model.BinaryResultsWrapper at 0x1cdd97cfa70>,
 'fitted_model_0': <statsmodels.discrete.discrete_model.BinaryResultsWrapper at 0x1cdd97cede0>,
 'fitted_model_1': <statsmodels.discrete.discrete_model.BinaryResultsWrapper at 0x1cdd97caed0>}

In [18]:
trial_itt.set_censor_weight_model(censor_event="censored", numerator="x2", denominator="x2+x1", pool_models="numerator", model_fitter=stats_glm_logit(save_path=os.path.join(path, "switch_models")))
trial_itt.show_censor_weights()

Optimization terminated successfully.
         Current function value: 0.682694
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.623353
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.634002
         Iterations 5


{'numerator': '1 - censored ~ x2',
 'denominator': '1 - censored ~ x2+x1',
 'pool_numerator': True,
 'pool_denominator': False,
 'model_fitter': 'te_stats_glm_logit',
 'fitted_model': <statsmodels.discrete.discrete_model.BinaryResultsWrapper at 0x1cdd97ce630>,
 'fitted_model_0': <statsmodels.discrete.discrete_model.BinaryResultsWrapper at 0x1cdd3b83a70>,
 'fitted_model_1': <statsmodels.discrete.discrete_model.BinaryResultsWrapper at 0x1cdd97cf4d0>}

#### 4. Calculate Weights
Next we need to fit the individual models and combine them into weights. This is done with calculate_weights().

In [19]:
trial_pp.calculate_weights()
trial_itt.calculate_weights()
print(trial_pp.data.columns)


Calculating switch weights...
Calculating censor weights...
Calculating censor weights...
Index(['id', 'period', 'treatment', 'x1', 'x2', 'x3', 'x4', 'age', 'age_s',
       'outcome', 'censored', 'eligible', 'previous_treatment', 'switch_prob',
       'switch_weight', 'censor_prob', 'censor_weight', 'final_weight'],
      dtype='object')


In [20]:
trial_pp.show_weight_models()

## Weight Models for Informative Censoring
#
#
## Model: P(censored = 0 | X) for numerator
#
Optimization terminated successfully.
         Current function value: 0.267425
         Iterations 7
              Coef.  Std.Err.          z     P>|z|    [0.025    0.975]
Intercept  2.205875  0.165376  13.338511  0.000000  1.881743  2.530007
x1         0.701948  0.307264   2.284511  0.022342  0.099721  1.304174
x2        -0.470645  0.137497  -3.422940  0.000619 -0.740134 -0.201155
#
## Model: P(censored = 0 | X, previous treatment = 0) for denominator
#
Optimization terminated successfully.
         Current function value: 0.287706
         Iterations 7
              Coef.  Std.Err.         z     P>|z|    [0.025    0.975]
Intercept  1.861908  0.215667  8.633235  0.000000  1.439207  2.284608
x1         1.225127  0.402734  3.042029  0.002350  0.435784  2.014471
x2        -0.479630  0.185757 -2.582034  0.009822 -0.843706 -0.115554
#
## Model: P(censored = 0 | X, previous treatment = 1) for denom

In [21]:
trial_itt.show_weight_models()

## Weight Models for Informative Censoring
#
#
## Model: P(censored = 0 | X) for numerator
#
Optimization terminated successfully.
         Current function value: 0.267425
         Iterations 7
              Coef.  Std.Err.          z     P>|z|    [0.025    0.975]
Intercept  2.205875  0.165376  13.338511  0.000000  1.881743  2.530007
x1         0.701948  0.307264   2.284511  0.022342  0.099721  1.304174
x2        -0.470645  0.137497  -3.422940  0.000619 -0.740134 -0.201155
#
## Model: P(censored = 0 | X, previous treatment = 0) for denominator
#
Optimization terminated successfully.
         Current function value: 0.287706
         Iterations 7
              Coef.  Std.Err.         z     P>|z|    [0.025    0.975]
Intercept  1.861908  0.215667  8.633235  0.000000  1.439207  2.284608
x1         1.225127  0.402734  3.042029  0.002350  0.435784  2.014471
x2        -0.479630  0.185757 -2.582034  0.009822 -0.843706 -0.115554
#
## Model: P(censored = 0 | X, previous treatment = 1) for denom

### 5. Specify Outcome Model
Now we can specify the outcome model. Here we can include adjustment terms for any variables in the dataset. The numerator terms from the stabilised weight models are automatically included in the outcome model formula.

In [ ]:
trial_pp.set_outcome_model()  
print(trial_pp.show_outcome_model())  

                 Generalized Linear Model Regression Results                  
Dep. Variable:                outcome   No. Observations:                  725
Model:                            GLM   Df Residuals:                      721
Model Family:                Binomial   Df Model:                            3
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -55.606
Date:                Sat, 01 Mar 2025   Deviance:                       111.21
Time:                        22:39:28   Pearson chi2:                     751.
No. Iterations:                     8   Pseudo R-squ. (CS):           0.003803
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.5619      1.470     -3.104      0.0

C:\Users\Rhenz\AppData\Roaming\Python\Python312\site-packages\statsmodels\base\model.py:130: ValueWarning: unknown kwargs ['weights']
  warnings.warn(msg, ValueWarning)
